In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd


train = pd.read_csv("/content/drive/MyDrive/Implicit/Ciao/trainImplicit.csv")
train = train[["userId","productId","rating"]]
train

userId  productId  rating
0          692      38404       1
1          850       6956       1
2          245      17754       1
3         1084      18155       1
4         1401      40830       1
...        ...        ...     ...
181908    2041      63223       1
181909     584      36373       1
181910      45       4765       1
181911    6270      47274       1
181912    2658      68970       1

[181913 rows x 3 columns]

In [ ]:
print(max(train["userId"]))
print(max(train["productId"]))

7372
91090


In [ ]:
import numpy as np
from scipy.sparse import csc_matrix
mat_data = csc_matrix((7373,91091), dtype=np.int8)
mat_data = mat_data.tolil()
print(mat_data.shape)
for uid,pid,_ in train.values:
  mat_data[uid,pid] = 1
#mat_data = mat_data.tocoo()

(7373, 91091)


In [ ]:
train_popularity = np.loadtxt("/content/drive/MyDrive/Implicit/Ciao/train_popularity.txt")

In [ ]:
import pickle

with open('/content/drive/MyDrive/Implicit/Ciao/user_neighbor_items_test.pickle', 'rb') as handle:
    user_neighbor_items = pickle.load(handle)

In [ ]:
mat_data_coo = mat_data.tocoo(copy=True)
mat_data_lil = mat_data_coo.tolil(copy=True)
mat_data_todok = mat_data_coo.todok(copy=True)

In [ ]:
class Sampler(object):

    def __init__(self, lil_record, dok_record, neg_sample_rate,num_user,num_item):

        self.lil_record = lil_record
        self.record = list(dok_record.keys())
        self.neg_sample_rate = neg_sample_rate
        self.n_user = num_user
        self.n_item = num_item
    

    def sample(self, index):

        user, pos_item = self.get_pos_user_item(index)

        users = np.full(self.neg_sample_rate, user, dtype=np.int64)
        items_pos = np.full(self.neg_sample_rate, pos_item, dtype=np.int64)
        items_neg = self.generate_negative_samples(user, pos_item=pos_item)

        return users, items_pos, items_neg
    
    def get_pos_user_item(self, index):

        user = self.record[index][0]
        pos_item = self.record[index][1]

        return user, pos_item
    
    def generate_negative_samples(self, user, **kwargs):

        negative_samples = np.full(self.neg_sample_rate, -1, dtype=np.int64)

        user_pos = self.lil_record.rows[user]
        for count in range(self.neg_sample_rate):

            item = np.random.randint(self.n_item)
            while item in user_pos or item in negative_samples:
                item = np.random.randint(self.n_item)
            negative_samples[count] = item
        
        return negative_samples

In [ ]:
from torch.utils.data import Dataset, DataLoader

class FactorizationDataset(Dataset):

    def __init__(self, mat_data_coo,mat_data_lil,mat_data_todok,train_popularity):
        self.make_sampler(mat_data_coo,mat_data_lil,mat_data_todok,train_popularity)

    def make_sampler(self, mat_data_coo,mat_data_lil,mat_data_todok,train_popularity):
        train_coo_record = mat_data_coo
        train_lil_record = mat_data_lil
        train_dok_record = mat_data_todok

        self.make_sampler_core(train_lil_record, train_dok_record)

    def __len__(self):

        return len(self.sampler.record)

    def __getitem__(self, index):

        raise NotImplementedError

class DICEFactorizationDataset(FactorizationDataset):

    def __init__(self, mat_data_coo,mat_data_lil,mat_data_todok,train_popularity):

        super(DICEFactorizationDataset, self).__init__(mat_data_coo,mat_data_lil,mat_data_todok,train_popularity)

    def make_sampler(self, mat_data_coo,mat_data_lil,mat_data_todok,train_popularity):

        self.popularity = train_popularity

        train_coo_record = mat_data_coo
        train_lil_record = mat_data_lil
        train_dok_record = mat_data_todok

        self.sampler = Sampler(train_lil_record, train_dok_record, 4,7373,91091)

    def __len__(self):

        return len(self.sampler.record)

    def __getitem__(self, index):

        if index < len(self.sampler.record):
            users, items_p, items_n = self.sampler.sample(index)
        
        return users, items_p, items_n

    def adapt(self, epoch, decay):
        self.sampler.adapt(epoch, decay)

In [ ]:
dataset = DICEFactorizationDataset(mat_data_coo,mat_data_lil,mat_data_todok,train_popularity)
x = DataLoader(dataset, batch_size=512, shuffle=True, num_workers=1, drop_last=False)

In [ ]:
#!pip install dgl
!pip install dgl-cu102 -f https://data.dgl.ai/wheels/repo.html

Looking in links: https://data.dgl.ai/wheels/repo.html


In [ ]:
import scipy.sparse
import math
import numpy as np

import torch
import torch.nn as nn
from torch.nn.parameter import Parameter
import torch.nn.functional as F
import dgl.function as fn
import dgl


from tqdm import tqdm

import random

class LGConv(nn.Module):

    def __init__(self,
                 in_feats,
                 out_feats,
                 k=1,
                 cached=False,
                 bias=True,
                 norm=None):
        super(LGConv, self).__init__()
        self._cached = cached
        self._cached_h = None
        self._k = k
        self.norm = norm

    def forward(self, graph, feat):

        graph = graph.local_var()
        if self._cached_h is not None:
            feat = self._cached_h
        else:
            # compute normalization
            degs = graph.in_degrees().float().clamp(min=1)
            norm = torch.pow(degs, -0.5)
            norm = norm.to(feat.device).unsqueeze(1)
            # compute (D^-1 A^k D)^k X
            for _ in range(self._k):
                feat = feat * norm
                graph.ndata['h'] = feat
                graph.update_all(fn.copy_u('h', 'm'),
                                 fn.sum('m', 'h'))
                feat = graph.ndata.pop('h')
                feat = feat * norm

            if self.norm is not None:
                feat = self.norm(feat)

            # cache feature
            if self._cached:
                self._cached_h = feat

        return feat

class LGN(nn.Module):

    def __init__(self, num_users, num_items, embedding_size, num_layers, dropout,graph_path):

        super(LGN, self).__init__()

        self.n_user = num_users
        self.n_item = num_items
        self.init_graph(graph_path)
        self.graph = self.graph.to("cuda")

        self.embeddings = Parameter(torch.FloatTensor(num_users + num_items, embedding_size))

        self.layers = nn.ModuleList()
        for _ in range(num_layers):
            self.layers.append(LGConv(embedding_size, embedding_size, 1))

        self.dropout = dropout

        self.init_params()

    def init_graph(self, graph_path):

        self.coo_adj_graph = scipy.sparse.load_npz(graph_path)
        self.graph = dgl.DGLGraph()

        num_nodes = self.coo_adj_graph.shape[0]
        self.graph.add_nodes(num_nodes)
        self.graph.ndata['feature'] = torch.arange(num_nodes)

        self.graph.add_edges(self.coo_adj_graph.row, self.coo_adj_graph.col)
        self.graph.add_edges(self.graph.nodes(), self.graph.nodes())

        print(self.graph.nodes())
        #self.graph.readonly()

    def bpr_loss(self, p_score, n_score):
      return -torch.mean(torch.log(torch.sigmoid(p_score - n_score)))
    
    def init_params(self):

        stdv = 1. / math.sqrt(self.embeddings.size(1))
        self.embeddings.data.uniform_(-stdv, stdv)

    def pair_forward(self, user, item_p, item_n, training=True):

        features = [self.embeddings]
        h = self.embeddings
        for layer in self.layers:
            h = layer(self.graph, h)
            h = F.dropout(h, p=self.dropout, training=training)
            features.append(h)

        features = torch.stack(features, dim=2)
        features = torch.mean(features, dim=2)

        item_p = item_p + self.n_user
        item_n = item_n + self.n_user

        user = features[user]
        item_p = features[item_p]
        item_n = features[item_n]

        p_score = torch.sum(user * item_p, 2)
        n_score = torch.sum(user * item_n, 2)

        return self.bpr_loss(p_score, n_score)

    def get_embeddings(self):

        features = [self.embeddings]
        h = self.embeddings
        for layer in self.layers:
            h = layer(self.graph, h)
            features.append(h)

        features = torch.stack(features, dim=2)
        features = torch.mean(features, dim=2)

        users = features[:self.n_user]
        items = features[self.n_user:]

        return items.detach().cpu().numpy().astype('float32'), users.detach().cpu().numpy().astype('float32')

In [ ]:
from tqdm import tqdm
import torch

f = "/content/drive/MyDrive/Implicit/Ciao/train_adj_DICE.npz"
model = LGN(7373, 91091, 64,2,0.2,f)
model = model.cuda()
#model.load_state_dict(torch.load("/content/drive/MyDrive/Implicit/Epinions/Models/DICE_20_0.001",map_location=torch.device('cpu')))
#model.load_state_dict(torch.load("/content/drive/MyDrive/Implicit/Epinions/Models/DICE_"+str(3)+"_"+str(1e-3)))
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)
min_loss = 1e+5
for epoch in range(0,500):
  running_loss = 0
  total_loss = 0
  for batch_count, sample in enumerate(tqdm(x)):
    user, item_p, item_n = sample
    user = user.cuda()
    item_p = item_p.cuda()
    item_n = item_n.cuda()
    loss = model.pair_forward(user, item_p, item_n)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    total_loss += loss.item()
  if(round(total_loss/(batch_count+1),2)<round(min_loss/(batch_count+1),2)):
    min_loss = total_loss
    breakout = 0 
  else:
    breakout+=1
  if(breakout==3):
    break
  if(epoch%3==0):
      file = "/content/drive/MyDrive/Implicit/Ciao/Models/LGN_TestSoc_"+str(epoch)+"_"+str(1e-3)
      torch.save(model.state_dict(),f=file)
      print("Model Saved")
  print("BPR Loss is ", round(total_loss/(batch_count+1),2), " for epoch ", epoch)

/usr/local/lib/python3.7/dist-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)


tensor([    0,     1,     2,  ..., 98461, 98462, 98463])


100%|██████████| 354/354 [00:22<00:00, 15.81it/s]


Model Saved
BPR Loss is  0.53  for epoch  0


100%|██████████| 354/354 [00:22<00:00, 15.80it/s]


...

Model Saved
BPR Loss is  0.01  for epoch  12


100%|██████████| 354/354 [00:22<00:00, 15.79it/s]


BPR Loss is  0.01  for epoch  13


 29%|██▉       | 104/354 [00:06<00:16, 15.39it/s]


KeyboardInterrupt: ignored

In [ ]:
from tqdm import tqdm
import torch

f = "/content/drive/MyDrive/Implicit/Epinions/train_adj.npz"
#model = LGN(37952, 119015, 64,2,0.2,f)
#model = model.cuda()
#model.load_state_dict(torch.load("/content/drive/MyDrive/Implicit/Epinions/Models/LGNDice_MMD_Test_"+str(0)+"_"+str(1e-3)))
#model.graph = model.graph.to("cuda")
#model.load_state_dict(torch.load("/content/drive/MyDrive/Implicit/Epinions/Models/LGN_TestSoc_"+str(12)+"_"+str(1e-3)))
item_emb,user_emb = model.get_embeddings()

In [ ]:
print(user_emb.shape)
print(item_emb.shape)

In [ ]:
import os
import math
import warnings
warnings.filterwarnings("ignore")
def _sample_negative(item_map,ratings):
        random.seed(10)
        """return all negative items & 100 sampled negative items"""
        interact_status = ratings.groupby('userId')['productId'].apply(set).reset_index().rename(
            columns={'productId': 'interacted_items'})
        interact_status['negative_samples'] = interact_status['interacted_items'].apply(lambda x: random.sample(set(list(range(len(item_map)))) - x, 40))
        return interact_status[['userId', 'negative_samples']]
        #interact_status['negative_items'] = interact_status['interacted_items'].apply(lambda x: set(list(range(len(item_map)))) - x)
        #interact_status['negative_samples'] = interact_status['negative_items'].apply(lambda x: random.sample(x, 10))
        #return interact_status[['userId', 'negative_items', 'negative_samples']]

def cal_hit_ratio(subjects,topk=10):
        """Hit Ratio @ top_K"""
        full, top_k = subjects,topk
        top_k = full[full['rank']<=top_k]
        test_in_top_k =top_k[top_k['test_item'] == top_k['item']]  # golden items hit in the top_K items
        return len(test_in_top_k) * 1.0 / full['user'].nunique()

def cal_ndcg(subjects,topk=10):
        full, top_k = subjects,topk
        top_k = full[full['rank']<=top_k]
        test_in_top_k =top_k[top_k['test_item'] == top_k['item']]
        test_in_top_k['ndcg'] = test_in_top_k['rank'].apply(lambda x: math.log(2) / math.log(1 + x)) # the rank starts from 1
        return test_in_top_k['ndcg'].sum() * 1.0 / full['user'].nunique()



#model.load_state_dict(torch.load("/content/drive/MyDrive/Copy of Epinions/Models_Final/basic_model_without_network"+str(180)+"_"+str(1e-2),pickle_module=dill)['state_dict'])
#path = "/content/drive/MyDrive/Copy of Epinions/testData_Final/"
#path = "/content/drive/MyDrive/Copy of Epinions/testData_Final/ratings_data_test.csv"
#path = "/content/drive/MyDrive/Copy of Epinions/Ciao/"
#path = "/content/drive/MyDrive/Copy of Epinions/Ciao/ratings_data_test.csv"
#files = os.listdir(path)
path = "/content/drive/MyDrive/Implicit/Epinions/"
model.eval()

files = os.listdir(path)
for f in files:
  if("test" in f and f in ["testImplicit.csv","ratings_data_test_implicit1_2_0.csv","ratings_data_test_implicit1_3_0.csv","ratings_data_test_implicit1_4_0.csv"]):
    if(f=="test.csv"):
      continue
    print(f)  
    test = pd.read_csv(path+f)
    test = test[["userId","productId","rating"]]
    test_negatives = _sample_negative(list(test["productId"].unique()),test)
    a = pd.merge(test, test_negatives[['userId', 'negative_samples']], on='userId')
    del test_negatives
    test_users,test_items,ratings,negative_users, negative_items,negative_ratings =[],[],[],[],[],[]
    for row in a.itertuples():
      test_users.append(int(row.userId))
      test_items.append(int(row.productId))
      ratings.append(float(row.rating))
      for i in range(len(row.negative_samples)):
          negative_users.append(int(row.userId))
          negative_items.append(int(row.negative_samples[i]))
          negative_ratings.append(float(0))  # negative samples get 0 rating
    del a
    tu,ti,tr,nu,ni,nr = torch.LongTensor(test_users), torch.LongTensor(test_items), torch.DoubleTensor(ratings), torch.LongTensor(negative_users),torch.LongTensor(negative_items),torch.DoubleTensor(negative_ratings)
    predr = []
    neg_predr = []
    for i in range(len(tu)):
      user_vec = torch.FloatTensor(user_emb[tu[i]])
      item_vec = torch.FloatTensor(item_emb[ti[i]])
      score = torch.sigmoid(torch.matmul(user_vec,item_vec.T))
      predr.append(score.cpu().detach().numpy())
    for i in range(len(nu)):
      user_vec = torch.FloatTensor(user_emb[nu[i]])
      item_vec = torch.FloatTensor(item_emb[ni[i]])
      score = torch.sigmoid(torch.matmul(user_vec,item_vec.T))
      neg_predr.append(score.cpu().detach().numpy())
    subjects = [tu,ti,predr,nu,ni,neg_predr]
    test_users, test_items, test_scores = subjects[0], subjects[1], subjects[2] 
    neg_users, neg_items, neg_scores = subjects[3], subjects[4], subjects[5]
    # the golden set
    test = pd.DataFrame({'user': test_users,
                        'test_item': test_items,
                        'test_score': test_scores})
    # the full set
    full = pd.DataFrame({'user':  np.append(neg_users,test_users),
                      'item':  np.append(neg_items,test_items),
                      'score':  np.append(neg_scores,test_scores)})
    full = pd.merge(full, test, on=['user'], how='left')
    # rank the items according to the scores for each user
    full['rank'] = full.groupby('user')['score'].rank(method='first', ascending=False)
    full.sort_values(['user', 'rank'], inplace=True)
    subjects = full
    print("Hit Ratio is: ",cal_hit_ratio(subjects,5))
    print("NDCG Value is: ",cal_ndcg(subjects,5))
    del full

testImplicit.csv
